<div style='color: #690027;' markdown="1">
    <h1>Diep neuraal netwerk</h1> 
</div>

Voer onderstaande codecel uit om van de methodes in deze notebook gebruik te kunnen maken.

In [ ]:
import imp
with open('../.scripts/diep_neuraal_netwerk.py', 'rb') as fp:
    diep_neuraal_netwerk = imp.load_module('.scripts', fp, '../.scripts/diep_neuraal_netwerk.py', ('.py', 'rb', imp.PY_SOURCE))

In deze notebook gaan we stap per stap een diep neuraal netwerk opbouwen dat in staat is een onderscheid te maken tussen een afbeelding met een stoma en een afbeelding zonder stoma, dit wordt ook wel een classificatie probleem genoemd waarbij een afbeelding wordt geclassificeerd als 'wel een stoma' of 'geen stoma'.

<div style='color: #690027;' markdown="1">
    <h2>1. De data</h2> 
</div>

Een diep neuraal netwerk leert een invoer af te beelden op een uitvoer door beschikbare data te verwerken. Deze data bestaat uit invoer en de daarbij horende verwachte uitvoer van het netwerk. Voor het stomata probleem bestaat de beschikbare data uit microscopische foto's van bladeren van verschillende soorten planten. Deze foto's zijn opgedeeld in 2 klassen die de uitvoer van het netwerk voorstellen. De foto's met een stoma in het midden behoren tot de klasse 'Stoma', de foto's zonder of met een gedeeltelijke stoma behoren tot de klasse 'Geen stoma'. Alle foto's hebben een lengte en breedte van 120 pixels en <b>er is experimenteel bepaald dat er 6 keer meer 'Geen stoma' foto's nodig zijn dan 'Stoma' foto's voor het beste resultaat</b>. Volgende afbeelding geeft een voorbeeld van de beschikbare invoer en de daarbij horende uitvoer.

<img src="../.images/IntroductieDeepLearning/training_data.jpg"/>

De beschikbare data wordt het best opgedeeld in 3 sets:
<ul>
    <li><b>Training set</b>: Dit is de data die gebruikt wordt om het model te trainen.<br>76 740 'geen stoma' + 12 790 'stoma' = 89 530 training afbeeldingen</li>
    <li><b>validatie set</b>: Deze data wordt gebruikt om te kijken hoe goed het model presteert op data die het nog niet gezien heeft. Op basis van deze data wordt het netwerk bijgeschaafd om betere resultaten te bekomen.<br>28 866 'geen stoma' + 4 811 'stoma'  = 33 677 validatie afbeeldingen</li>
    <li><b>Test set</b>: Na het trainen met de training set en bijschaven van het netwerk aan de hand van de validatie set wordt het netwerk nog 1 keer geëvalueerd met de test set om de eindscore te berekenen.<br>55 182 'geen stoma' + 9 197 'stoma' = 64 379 test afbeeldingen</li>
</ul>

<div style='color: #690027;' markdown="1">
    <h2>2. De netwerk architectuur</h2> 
</div>

Een diep neuraal netwerk bestaat uit verschillende opeenvolgende lagen die de invoer omzetten in de uitvoer. Volgende afbeelding stelt een basis netwerk voor. De volgende sub paragrafen beschrijven elk een deel van het netwerk. Het cijfer 1 op de afbeelding staat dus voor paragraaf 2.1, het cijfer 2 voor paragraag 2.2, enz.

<img src="../.images/IntroductieDeepLearning/vb_netwerk.jpg"/>

<div style='color: #690027;' markdown="1">
    <h2>2.1: Invoer</h2> 
</div>

Zoals eerder vermeld is de invoer van het netwerk een microscopische foto van een blad waarop wel of juist geen stoma te zien is. Deze foto heeft een lengte en een breedte van 120 pixels en zal worden voorgesteld als een tensor ([Tensoren](../IntroductiePython/Tensoren.ipynb)) met dimensie 120x120x3 zodat hij verwerkt kan worden door het netwerk.

<div style='color: #690027;' markdown="1">
    <h2>2.2: Convolutionele lagen</h2> 
</div>

Om relevante patronen te ontdekken in de ingevoerde afbeelding (voorgesteld als tensor) worden convolutionele lagen gebruikt. Een convolutionele laag zal een tweede tensor (<b>filter</b>) met een zelfgekozen x en y dimensie en dezelfde z-dimensie als de invoer op de afbeelding leggen en een wiskundige berekening ([convolutie](../IntroductieDeeplearning/Convolutie.ipynb)) uitvoeren die 1 getal terug geeft. Hierna zal deze filter een zelfgekozen aantal pixels opschuiven en wordt het volgende getal berekend, hierdoor krijg je een tweedimensionale tensor die informatie bevat over waar het patroon van de filter voorkomt in de invoer. Volgende afbeelding toont deze operatie op een afbeelding met lengte en breedte 5 en diepte 3 (RGB) en met een filter met lengte en breedte 3 die steeds 1 positie opschuift.

<img src="../.images/IntroductieDeepLearning/conv_operation.jpg"/>

Een convolutionele laag zal meestal niet 1 maar meerdere filters gebruiken om verschillende patronen te herkennen. Hierdoor krijg je per filter die over de afbeelding glijdt een tweedimensionale uitvoer met dezelfde lengte en breedte. Door deze uitvoeren te combineren krijg je opnieuw een driedimensionale tensor waarbij de z-dimensie gelijk is aan het aantal filters, dit wordt een <b>feature map</b> genoemd. De waarden van deze filters worden aangepast tijdens het trainen van het model om relevante patronen te herkennen. In de afbeelding van ons basis netwerk stelt het getal onder de convolutionele laag (hier 32) het aantal filters en dus de z-dimensie van de uitvoer tensor van de convolutionele laag voor.

<img src="../.images/IntroductieDeepLearning/conv_layer_2.jpg"/>

Het netwerk dat we samenstellen voor de classificatie van stomata zal filters gebruiken met lengte en breedte 3 en deze filters zullen steeds 1 positie per keer opschuiven.

<div style='color: #690027;' markdown="1">
    <h2>2.3: Max-pooling</h2> 
</div>

De convolutionele lagen worden afgewisseld met max-pooling operaties, deze operaties gaan uit een venster met een zelfgekozen grootte enkel de grootste waarde overhouden. We kiezen voor de grootste waarde omdat deze aangeeft waar de filter het meest aanwezig was in de invoer. Volgende afbeelding geeft een voorbeeld van de max-pooling operatie.

<img src="../.images/IntroductieDeepLearning/max_pooling.jpg"/>

Het doel van max-pooling operaties is twee delig, eenderzijds willen we de grootte van de uitvoer van de convolutionele lagen verkleinen (Waarom dit nodig is wordt beschreven in de volgende paragraaf). Anderzijds willen we meer informatie bekomen over een groter vlak van de ingevoerde afbeelding. Stel je even voor dat de max-pooling operaties er niet zouden zijn, dan zou na twee opeenvolgende convolutionele lagen een veld met lengte en breedte 1 slechts informatie bevatten over een veld met lengte en breedte 5 van de ingevoerde afbeelding, dit is vaak niet genoeg om belangrijke kenmerken van de afbeelding te herkennen (zou je zelf een afbeelding herkennen aan een aantal pixels?). Volgende afbeelding toont wat er gebeurt met twee convolutionele lagen (met steeds maar 1 filter) zonder max-pooling operaties. Het rode veld in de laatste feature map bevat enkel informatie uit het rode veld uit de ingevoerde afbeelding.

<img src="../.images/IntroductieDeepLearning/conv_no_max_pooling.jpg"/>

In ons netwerk gebruiken we max-pooling in velden met een lengte en breedte van 2, hierdoor wordt de grootte van de feature maps gehalveerd na een max-pooling operatie.

<div style='color: #690027;' markdown="1">
    <h2>2.4: Feedforward lagen</h2> 
</div>

Om de effectieve classificatie van de afbeelding uit te voeren gebruiken we feedforward lagen. Deze lagen bestaan uit neuronen ([StomataZonSchaduw](../IntroductieMachineLearning/StomataZonSchaduw.ipynb)) waarbij elk neuron is verbonden met elke andere neuron van de volgende feedforward laag. Al deze verbindingen tussen de neuronen van de opeenvolgende lagen hebben een bepaalde waarde die aangeeft in welke mate de uitvoer van 1 neuron de volgende neuron activeert, deze waarden worden ook wel de <b>gewichten</b> van het netwerk genoemd. Een netwerk leert door deze gewichten aan te passen afhankelijk van de training data. Hoe meer neuronen, hoe meer informatie het netwerk kan opslaan. Te veel neuronen is echter ook niet altijd goed (zie notebook [Overfitting](../IntroductieDeepLearning/Overfitting.ipynb)). Volgende afbeelding toont een netwerk dat alleen uit feedforward lagen bestaat, de cirkels zijn de neuronen, de verbindingen zijn de gewichten.

<img src="../.images/IntroductieDeepLearning/ffn.jpg"/>

Tot nu toe is onze ingevoerde afbeelding door de convolutionele lagen en de max-pooling operaties omgezet in een driedimensionale tensor die informatie bevat over verschillende relevante patronen in die afbeelding. Vooraleer deze data als invoer van de feedforward lagen kan dienen moet de driedimensionale tensor worden omgezet in een ééndimensionale tensor, dit doen we met een <b>flatten</b> operatie. Volgende afbeelding toont hoe de flatten operatie een feature map met dimensie 3x3x1 omzet om als invoer te dienen voor de feedforward lagen.

<img src="../.images/IntroductieDeepLearning/flatten.jpg"/>

Nu is ook meteen duidelijk waarom de uitvoer van de convolutionele lagen niet te groot mag zijn. Neem bijvoorbeeld een flatten operatie op een feature map met dimensie 100x100x64, dit geeft 640000 verschillende invoeren voor de feedforward lagen. Als de eerste feedforward laag dan nog eens 64 neuronen bevat zijn er tussen deze twee lagen alleen al bijna 41 miljoen gewichten die het netwerk moet leren.

In onze voorstelling van het diep neuraal netwerk wordt een feedforward laag voorgesteld als een paars balkje. Het getal onder het balkje stelt het aantal neuronen in deze laag voor.

<div style='color: #690027;' markdown="1">
    <h2>2.5: Uitvoer</h2> 
</div>

De laatste laag bestaat uit slechts 1 neuron en geeft een getal tussen 0 en 1 terug, dit is de voorspelling van het netwerk waarbij 0 staat voor 'geen stoma' en 1 voor 'stoma'. Hoe dichter de uitvoer bij 1 ligt hoe zekerder het netwerk is dat er op de ingevoerde afbeelding een stoma te zien is.

<div style='color: #690027;' markdown="1">
    <h2>2.6. Kies je netwerk architectuur</h2> 
</div>

Nu je de verschillende onderdelen van een diep neuraal netwerk begrijpt kan je zelf je netwerk samenstellen om de stomata classificatie uit te voeren. Voer onderstaande codecel uit om de netwerk parameters te kiezen.

In [ ]:
diep_neuraal_netwerk.kies_netwerk_parameters()

Om je gekozen netwerk te visualiseren voer je volgende code cel uit. Ben je niet tevreden of wil je andere zaken uitproberen verander dan gerust de parameters en voer onderstaande codecel opnieuw uit.

In [ ]:
diep_neuraal_netwerk.toon_netwerk()

<div style='color: #690027;' markdown="1">
    <h2>3. Het netwerk trainen</h2> 
</div>

Nu de netwerkarchitectuur gekozen is moet dit netwerk getraind worden. Vooraleer we dit kunnen doen moeten er opnieuw een aantal keuzes worden gemaakt.

<div style='color: #690027;' markdown="1">
    <h2>3.1 Loss functie</h2> 
</div>

De loss functie is een functie die beschrijft hoe goed het netwerk presteert met de huidige gewichten. Hoe lager de waarde van de loss functie hoe dichter de huidige uitvoer van het netwerk bij de gewenste uitvoer ligt. Het netwerk zal een aantal afbeeldingen samen namen (<b>een batch</b>) en kijkt dan in welke mate deze juist geclassificeerd werden. In ons netwerk is er gekozen voor de loss functie <b>binary crossentropy</b>. Deze loss functie wordt vaak gebruikt bij classificatie problemen met 2 klassen (vandaar de 'binary'). We gaan hier niet in detail hoe deze loss functie juist werkt.

<div style='color: #690027;' markdown="1">
    <h2>3.2 Optimizer</h2> 
</div>

Een andere belangrijke keuze bij het trainen van een diep neuraal netwerk is de <b>optimizer</b>, deze bepaald op welke manier het netwerk leert. In ons netwerk is er gekozen om de <b>stochastic gradient descent (SGD)</b> optimizer te gebruiken omdat deze het minst complex is. Zoals de naam al doet vermoeden maakt SGD gebruik van [gradient descent](../IntroductieDeepLearning/GradientDescent.ipynb), de afgeleide van de loss functie wordt berekend om zo een minimum hiervoor te vinden. De 'stochastic' betekent dat de afgeleide van de loss niet wordt berekent met de volledige training set maar met slechts een willekeurig deel hiervan. Dit gaat een stuk sneller dan wanneer de afgeleide van de loss wordt berekent met de volledige training set.

<div style='color: #690027;' markdown="1">
    <h2>3.3 Learning rate</h2> 
</div>

De learning rate bepaald hoe groot de stap is om in het minimum te geraken, een kleine learning rate zal ervoor zorgen dat het netwerk traag leert en met een te grote learning rate zal het netwerk het minimum niet vinden. Volgende afbeelding toont het gradient descent werkt met 2 dimensies, bij een diep neuraal netwerk zijn er enorm veel gewichten en dus ook enorm veel dimensies wat veel moeilijker voor te stellen is door een figuur.

<img src="../.images/IntroductieDeepLearning/gradient_descent.jpg" width="500"/>

<div style='color: #690027;' markdown="1">
    <h2>3.4 Epochs</h2> 
</div>

Je kan ervoor kiezen hoeveel keer de volledige training data verwerkt wordt, dit wordt het aantal <b>epochs</b> genoemd. Vaak moet je verschillende epochs proberen om het beste resultaat te bekomen. Ons netwerk is steeds getraind met 20 epochs.

<div style='color: #690027;' markdown="1">
    <h2>3.4 Kies je training parameters</h2> 
</div>

Voer onderstaande codecel uit om de trainings parameters in te stellen (voor de optimizer is gekozen voor SGD).

In [ ]:
diep_neuraal_netwerk.kies_training_parameters()

<div style='color: #690027;' markdown="1">
    <h2>4. Resultaten</h2> 
</div>

Normaal gezien duurt het even om een netwerk te trainen, de netwerken die in dit voorbeeld gemaakt kunnen worden zijn echter op voorhand getraind en de resultaten zijn opgeslaan in een databank. Op deze manier kunnen we dus onmiddellijk kijken naar de prestaties van het gekozen netwerk. 

Er bestaan verschillende manieren om een netwerk te beoordelen. Enerzijds kan je kijken naar de waarde van de loss functie, anderzijds kan je ook kijken naar de <b>accuracy</b> of nauwkeurigheid van het netwerk, dit is het percentage van de samples waarvoor het netwerk de voorspelling juist heeft. Deze waarden kan je tijdens het trainen per epoch berekenen voor de training data en de validatie data. Na het trainen kan je deze waarden ook nog 1 maal berekenen op de test data om de finale beoordeling van het netwerk te bekomen.

Door volgende codecel uit te voeren zie je de training en validatie accuracy en loss voor het netwerk. De grafieken geven per epoch de training en de validatie accuracy of loss weer.

In [ ]:
diep_neuraal_netwerk.toon_grafiek()

Vaak zal je zien dat de training loss daalt terwijl de validation loss stijgt. Wanneer dit gebeurt is het netwerk aan het <b>overfitten</b>. Dit wil zeggen dat het netwerk te veel details van de training data van buiten leert en dus niet meer goed generaliseerd op data dat het nog nooit gezien heeft. Overfitting is 1 van de grootste problemen van een diep neuraal netwerk, daarom zal een volgende notebook ([Overfitting](../IntroductieDeepLearning/Overfitting.ipynb)) een aantal technieken uitleggen om overfitting tegen te gaan.

Het tegenovergestelde van overfitten is <b>underfitten</b>, dit wil zeggen dat het netwerk niet genoeg kan leren en dus slecht het relevante patroon in de data kan herkennen. Vaak is dit het geval bij een te simpel netwerk of een netwerk dat te kort werd getraind.

Een laatste geval is dat het netwerk niet leert. Om dit vast te stellen formuleer je een <b>baseline</b>, wat dit is is makkelijk uit te leggen aan de hand van ons voorbeeld. Onze dataset bevat 6 keer zo veel afbeeldingen zonder stoma dan afbeeldingen met een stoma. In totaal is dus 6/7 (85.7%) van de training set, validation set en test set een afbeelding zonder stoma. Wanneer het model dus alle afbeeldingen als 'geen stoma' aangeeft zal er dus al een accuracy van 85.7% zijn. Dit is de baseline die het model moet overtreffen vooraleer we kunnen zeggen dat het model iets heeft geleerd.

<div style='color: #690027;' markdown="1">
    <h2>4.1 Oefening</h2> 
</div>

Zoek een model dat overfit en een model dat niet leert.

<div style='color: #690027;' markdown="1">
    <h2>4.2 Voorspellingen</h2> 
</div>

Als extra tonen we nog een aantal voorspellingen van het netwerk. De voorspelling is een getal tussen 0 en 1 dat aangeeft hoe zeker het netwerk is dat er een stoma gevonden werd. 

Voer onderstaande codecel uit om de voorspellingen te zien.

In [ ]:
diep_neuraal_netwerk.toon_voorspellingen()

Er zijn ook afbeeldingen waar het model het moeilijk mee heeft. Wanneer het netwerk denkt dat een op een afbeelding met een stoma geen stoma staat spreekt men van een <b>false negative</b>. Omgekeerd, wanneer het netwerk denkt dat er om een afbeelding zonder stoma wel een stoma staat spreekt men van een <b>false positive</b>. 

Voer onderstaande codecel uit om enkele afbeeldingen te zien waarmee veel modellen het moeilijk hebben.

In [ ]:
diep_neuraal_netwerk.toon_slechte_voorspellingen()

Ben je tevreden over je netwerk dan gaan we het nog een laatste keer beoordelen aan de hand van de test set. Voeg volgende code-cel uit om het het uiteindelijke resultaat van je netwerk te zien.

In [ ]:
diep_neuraal_netwerk.toon_test_resultaten()